In [9]:
import http.client
import pprint
import json
from dotenv import load_dotenv
import os

load_dotenv()

LEAUGE_JSON = 'jleague_data.json'

In [ ]:
if __name__ == "__main__":
    with open(LEAUGE_JSON, 'r') as read_file:
        league_data = json.load(read_file)
    
    pprint.pprint(league_data)

In [4]:
# We needed to explore the league_data to suss out how to extract individual teams
i = 0
for team in league_data['response']:
    print(i)  
    print(team)
    i += 1
  

0
{'team': {'id': 279, 'name': 'Consadole Sapporo', 'code': 'SAP', 'country': 'Japan', 'founded': 1935, 'national': False, 'logo': 'https://media-4.api-sports.io/football/teams/279.png'}, 'venue': {'id': 955, 'name': 'Sapporo Dome', 'address': '1-3 Hitsujigaoka, Toyohira-ku', 'city': 'Sapporo', 'capacity': 53845, 'surface': 'grass', 'image': 'https://media-4.api-sports.io/football/venues/955.png'}}
1
{'team': {'id': 281, 'name': 'Kashiwa Reysol', 'code': 'KAS', 'country': 'Japan', 'founded': 1940, 'national': False, 'logo': 'https://media-4.api-sports.io/football/teams/281.png'}, 'venue': {'id': 965, 'name': 'SANKYO FRONTIER Kashiwa Stadium', 'address': '1-2-50 Kashiwa, Chiba', 'city': 'Kashiwa', 'capacity': 15900, 'surface': 'grass', 'image': 'https://media-4.api-sports.io/football/venues/965.png'}}
2
{'team': {'id': 282, 'name': 'Sanfrecce Hiroshima', 'code': 'HIR', 'country': 'Japan', 'founded': 1938, 'national': False, 'logo': 'https://media-4.api-sports.io/football/teams/282.png'}

In [6]:
# We found Kawasaki Frontale with counting
kawasaki_frontale = league_data['response'][11]
pprint.pprint(kawasaki_frontale)

{'team': {'code': 'KAW',
          'country': 'Japan',
          'founded': 1955,
          'id': 294,
          'logo': 'https://media-4.api-sports.io/football/teams/294.png',
          'name': 'Kawasaki Frontale',
          'national': False},
 'venue': {'address': '1-1 Todoroki, Nakahara-ku',
           'capacity': 27495,
           'city': 'Kawasaki',
           'id': 966,
           'image': 'https://media-4.api-sports.io/football/venues/966.png',
           'name': 'Kawasaki Todoroki Stadium',
           'surface': 'grass'}}


In [16]:
import http.client
conn = http.client.HTTPSConnection("v3.football.api-sports.io")
api_key = os.getenv('x-rapidapi-key')
squad_id = os.getenv('KAWASAKI_SQUAD_ID')
headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': api_key
    }

# We tried being cheeky and specifying the team ID via f-string, but that 
# did not go as planned. 
conn.request("GET", "/players/squads?team=294", headers=headers)

res = conn.getresponse()
data = res.read()

squad_data = data.decode("utf-8")
#pprint(data.decode("utf-8"))
pprint.pprint(squad_data)
                         

('{"get":"players\\/squads","parameters":{"team":"294"},"errors":[],"results":1,"paging":{"current":1,"total":1},"response":[{"team":{"id":294,"name":"Kawasaki '
 'Frontale","logo":"https:\\/\\/media-4.api-sports.io\\/football\\/teams\\/294.png"},"players":[{"id":32946,"name":"Jung '
 'Sung-Ryong","age":38,"number":1,"position":"Goalkeeper","photo":"https:\\/\\/media-4.api-sports.io\\/football\\/players\\/32946.png"},{"id":32943,"name":"S. '
 'Ando","age":33,"number":21,"position":"Goalkeeper","photo":"https:\\/\\/media-4.api-sports.io\\/football\\/players\\/32943.png"},{"id":315353,"name":"Y. '
 'Hayasaka","age":24,"number":22,"position":"Goalkeeper","photo":"https:\\/\\/media-4.api-sports.io\\/football\\/players\\/315353.png"},{"id":33785,"name":"N. '
 'Kamifukumoto","age":34,"number":99,"position":"Goalkeeper","photo":"https:\\/\\/media-4.api-sports.io\\/football\\/players\\/33785.png"},{"id":32953,"name":"K. '
 'Noborizato","age":33,"number":2,"position":"Defender","photo":"https:\

In [18]:
import http.client
conn = http.client.HTTPSConnection("v3.football.api-sports.io")
api_key = os.getenv('x-rapidapi-key')
squad_id = os.getenv('KAWASAKI_SQUAD_ID')
headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': api_key
    }

conn.request("GET", "/players/squads?team=294", headers=headers)

res = conn.getresponse()
#data = res.read()

# Persist the squad data in JSON so we don't take API hits unneccessarily
squad_data_json = json.load(res)
#pprint(data.decode("utf-8"))
pprint.pprint(squad_data_json)
                         

{'errors': [],
 'get': 'players/squads',
 'paging': {'current': 1, 'total': 1},
 'parameters': {'team': '294'},
 'response': [{'players': [{'age': 38,
                            'id': 32946,
                            'name': 'Jung Sung-Ryong',
                            'number': 1,
                            'photo': 'https://media-4.api-sports.io/football/players/32946.png',
                            'position': 'Goalkeeper'},
                           {'age': 33,
                            'id': 32943,
                            'name': 'S. Ando',
                            'number': 21,
                            'photo': 'https://media-4.api-sports.io/football/players/32943.png',
                            'position': 'Goalkeeper'},
                           {'age': 24,
                            'id': 315353,
                            'name': 'Y. Hayasaka',
                            'number': 22,
                            'photo': 'https://media-4.api-sports.

In [19]:
squad_data_json

{'get': 'players/squads',
 'parameters': {'team': '294'},
 'errors': [],
 'results': 1,
 'paging': {'current': 1, 'total': 1},
 'response': [{'team': {'id': 294,
    'name': 'Kawasaki Frontale',
    'logo': 'https://media-4.api-sports.io/football/teams/294.png'},
   'players': [{'id': 32946,
     'name': 'Jung Sung-Ryong',
     'age': 38,
     'number': 1,
     'position': 'Goalkeeper',
     'photo': 'https://media-4.api-sports.io/football/players/32946.png'},
    {'id': 32943,
     'name': 'S. Ando',
     'age': 33,
     'number': 21,
     'position': 'Goalkeeper',
     'photo': 'https://media-4.api-sports.io/football/players/32943.png'},
    {'id': 315353,
     'name': 'Y. Hayasaka',
     'age': 24,
     'number': 22,
     'position': 'Goalkeeper',
     'photo': 'https://media-4.api-sports.io/football/players/315353.png'},
    {'id': 33785,
     'name': 'N. Kamifukumoto',
     'age': 34,
     'number': 99,
     'position': 'Goalkeeper',
     'photo': 'https://media-4.api-sports.io/fo

In [23]:
# Used ChatGPT to teach me how to "fuzzy" match data in output HTML vs. 
# player data in the JSON output from API. Use these libraries!
pip install beautifulsoup4 fuzzywuzzy  python-Levenshtein



   ---------------------------------------- 0.0/101.1 kB ? eta -:--:--
   -------------------------------- ------- 81.9/101.1 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 101.1/101.1 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ------- -------------------------------- 0.4/1.8 MB 11.2 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.8 MB 13.5 MB/s eta 0:00:01
   -------------------------------- ------- 1.5/1.8 MB 15.8 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 14.5 MB/s eta 0:00:00


In [25]:
# This cell will take squad data from JSON, combined with HTML from FM, use 
# the HTML file as source of truth, look to "find" the player in the JSON,
# locate the associate image attribute for the player, download the image,
# and then save the downloaded image with the player's FM unique ID. 

import requests
import time
import random
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz

# JSON data (assuming it's stored in a variable)
#squad_data_json = {
    # ... Your JSON data here ...
#}

def download_image(image_url: str, uid: str) -> bool:
    response = requests.get(image_url)
    if response.status_code == 200:
        # Save the image with UID as the filename
        with open(f'{uid}.png', 'wb') as image_file:
            image_file.write(response.content)
        return True  # Return True to indicate success
    else:
        return False  # Return False to indicate failure

# Parse the HTML file
with open('players.html', 'r', encoding='utf-8') as html_file:
    soup = BeautifulSoup(html_file, 'html.parser')

# Extract player information from HTML
player_data_html = []
for row in soup.find_all('tr')[1:]:  # Skip the header row
    columns = row.find_all('td')
    if len(columns) >= 4:
        position = columns[0].get_text()
        name = columns[2].get_text()
        uid = columns[3].get_text()
        age = columns[7].get_text()
        player_data_html.append({'position': position, 'name': name, 'uid': uid, 'age': age})

# Create a dictionary to store the matched players and image URLs
matched_players = {}

# Iterate over player data from HTML
for player_html in player_data_html:
    # Define a threshold for fuzzy matching score
    # We HAD to lower this to 70 to match even simple, non-duplicated names. 
    matching_threshold = 70

    # Initialize variables for best matching player and image URL
    best_match = None
    best_match_score = 0
    best_image_url = None

    # Iterate over players in JSON data
    for player_json in squad_data_json['response'][0]['players']:
        # Calculate fuzzy matching score between HTML and JSON player names
        name_similarity = fuzz.partial_ratio(player_html['name'], player_json['name'])

        # Check if the player has the same position and similar age
        position_match = player_html['position'] == player_json['position']
        age_match = player_html['age'] == str(player_json['age'])

        # Calculate a weighted score based on name similarity, position, and age match
        total_score = (name_similarity * 0.6) + (position_match * 0.2) + (age_match * 0.2)

        # Update best match if the current player is a better match
        if total_score > best_match_score and name_similarity >= matching_threshold:
            best_match = player_json
            best_match_score = total_score

    # If a match is found, extract the image URL and download the image
    if best_match:
        best_image_url = best_match['photo']
        if download_image(best_image_url, player_html['uid']):
            # Download was successful, add player to matched players
            matched_players[player_html['uid']] = {'player_data': best_match, 'image_url': best_image_url}

            # Add a random time delay between 15 and 60 seconds
            delay_seconds = random.randint(15, 60)
            time.sleep(delay_seconds)

            # Continue with additional processing or actions
            print(f"Downloaded image for player with UID: {player_html['uid']}")   
            

# Now, 'matched_players' contains matched player information and image URLs


Downloaded image for player with UID: 5704105
Downloaded image for player with UID: 2000151924
